<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c2a9618ff3b4f68b805382f3dc47bd88212fc1481f0b0a7815d9e62c9593cb83
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-08 09:20:13
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.41 C
-------------------
Today PnL: -9.11 K (-0.06%)
Current PnL: -27.09 L (-17.5%)
CY Booked + Current PnL: -12.80 L (-8.27%)
-------------------
Total profit:  1.48 L
Total loss:  -28.57 L
-------------------
Total Booked + Current PnL: 14.18 L (11.09%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.12%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 91.72 L (64.97%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.14,-16.73,20.21,0.09,16959.0,-16865.0,83912.0,94.66,38.0,M-SC,2.38,253.0,-0.99,0.62,10.24,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.11,-12.13,13.83,0.02,21212.0,-21183.0,153376.0,133.82,53.0,M-SC,16.90,234.0,-1.00,1.13,33.41,OX40N,NTT,PAINTS
50,MASFIN,398.61,1.65,-0.21,22.31,22.05,21812.0,-210.0,97770.0,-14.18,61.0,H-SC,7.04,164.0,-0.01,0.72,42.12,XR,ATH,FINANCE
43,ITC,452.00,-0.17,-1.23,11.81,10.44,23346.0,-2460.0,197678.0,-41.15,48.0,X-LC,1.57,5.0,-0.11,1.45,4.27,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.42,4.90,14.58,20.19,26661.0,8537.0,182857.0,-17.91,44.0,X-MC,6.17,68.0,0.32,1.34,20.75,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,2.16,-42.93,205.17,74.15,191555.0,-70241.0,93364.0,-267.81,48.0,H-SC,20.68,149.0,-0.37,0.69,19.68,XR,NTT,CHEMICALS
50,MASFIN,398.61,1.65,-0.21,22.31,22.05,21812.0,-210.0,97770.0,-14.18,61.0,H-SC,7.04,164.0,-0.01,0.72,42.12,XR,ATH,FINANCE
18,COFFEEDAY,80.00,1.20,-42.95,137.11,35.27,88821.0,-48768.0,64781.0,-55.73,29.0,L-SC,13.02,270.0,-0.55,0.48,58.55,XR,NTT,HOTELS
68,STARHEALTH,761.00,1.06,-10.46,64.47,47.26,162277.0,-29414.0,251709.0,22.16,35.0,H-SC,13.78,171.0,-0.18,1.85,34.86,XY24,NTT,INSURANCE
24,EASEMYTRIP,26.40,1.04,-57.06,239.33,45.70,181692.0,-100898.0,75917.0,-21.23,50.0,M-SC,23.52,205.0,-0.56,0.56,9.42,XY24,NTT,TRAVEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIS,528.00,-1.69,-25.70,63.21,21.26,52143.0,-28540.0,82492.0,1944.04,42.0,H-SC,4.26,166.0,-0.55,0.61,11.46,OX40N,NTT,MISC
83,WHIRLPOOL,2270.00,-1.53,-25.07,142.78,81.92,122819.0,-28778.0,86020.0,-56.08,19.0,M-SC,8.12,236.0,-0.23,0.63,1.83,XR,NTT,DURABLES
38,INDIAMART,4810.62,-1.06,-1.68,110.25,106.72,133694.0,-2072.0,121264.0,-51.39,36.0,H-SC,1.34,139.0,-0.02,0.89,21.31,AR,ATH,MISC
55,QUESS,424.00,-1.05,-32.72,112.31,42.84,49120.0,-21270.0,43736.0,-55.92,32.0,X-SC,37.31,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
2,ABBOTINDIA,35195.00,-0.90,-4.29,22.69,17.44,32543.0,-6422.0,143425.0,-17.58,38.0,X-MC,4.34,63.0,-0.20,1.05,14.05,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,1.65,-0.21,22.31,22.05,21812.0,-210.0,97770.0,-14.18,61.0,H-SC,7.04,164.0,-0.01,0.72,42.12,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.14,-6.84,123.17,107.91,169418.0,-10096.0,137548.0,-24.20,30.0,M-SC,11.52,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.11,-12.13,13.83,0.02,21212.0,-21183.0,153376.0,133.82,53.0,M-SC,16.90,234.0,-1.00,1.13,33.41,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.14,-16.73,20.21,0.09,16959.0,-16865.0,83912.0,94.66,38.0,M-SC,2.38,253.0,-0.99,0.62,10.24,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.24,-23.69,48.69,13.47,100201.0,-63873.0,205794.0,-68.70,32.0,H-SC,2.81,158.0,-0.64,1.51,4.64,XY24,NTT,PAINTS
72,TATAELXSI,9161.00,-0.58,-25.24,76.60,32.03,75498.0,-33273.0,98562.0,-19.06,43.0,H-SC,7.72,148.0,-0.44,0.72,10.40,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.86,7.52,60.22,72.27,109713.0,12739.0,182187.0,-7.89,75.0,M-LC,2.79,99.0,0.12,1.34,14.16,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.86,7.52,60.22,72.27,109713.0,12739.0,182187.0,-7.89,75.0,M-LC,2.79,99.0,0.12,1.34,14.16,XR,NTT,IT
50,MASFIN,398.61,1.65,-0.21,22.31,22.05,21812.0,-210.0,97770.0,-14.18,61.0,H-SC,7.04,164.0,-0.01,0.72,42.12,XR,ATH,FINANCE
38,INDIAMART,4810.62,-1.06,-1.68,110.25,106.72,133694.0,-2072.0,121264.0,-51.39,36.0,H-SC,1.34,139.0,-0.02,0.89,21.31,AR,ATH,MISC
37,IEX,219.00,-0.38,-1.76,51.28,48.63,100734.0,-3515.0,196439.0,-34.58,55.0,H-SC,14.09,136.0,-0.03,1.44,9.59,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.20,-1.58,37.81,35.63,77987.0,-3315.0,206259.0,-15.41,42.0,H-MC,3.15,119.0,-0.04,1.51,14.83,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.21,-21.68,89.56,48.45,140636.0,-43476.0,157030.0,-27.39,13.0,X-MC,10.56,64.0,-0.31,1.15,0.21,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.32,-37.83,117.46,35.20,93968.0,-48670.0,80000.0,4.84,26.0,X-SC,14.98,92.0,-0.52,0.59,0.37,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.25,-11.92,38.04,21.58,46955.0,-16704.0,123435.0,-55.63,26.0,X-MC,5.33,56.0,-0.36,0.91,9.28,X40N,ATH,IT
54,PGHH,17907.65,-0.14,-6.11,42.46,33.76,80058.0,-12270.0,188550.0,-33.72,29.0,X-MC,5.07,57.0,-0.15,1.38,0.00,X40,ATH,FMCG
46,JSWINFRA,342.00,-0.69,-9.91,27.90,15.23,50358.0,-19852.0,180495.0,-22.98,30.0,X-MC,7.49,66.0,-0.39,1.32,19.15,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-0.51,-20.40,92.84,53.50,222985.0,-61569.0,240182.0,-61.46,35.0,X-MC,1.37,58.0,-0.28,1.76,6.68,XY24,NTT,FMCG
43,ITC,452.0,-0.17,-1.23,11.81,10.44,23346.0,-2460.0,197678.0,-41.15,48.0,X-LC,1.57,5.0,-0.11,1.45,4.27,X40,NTT,FMCG
3,ACC,3906.0,0.49,-24.06,116.15,64.15,209895.0,-57241.0,180710.0,-55.44,40.0,X-SC,1.66,82.0,-0.27,1.33,1.37,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.75,-79.82,518.52,24.79,263476.0,-201043.0,50813.0,-80.65,47.0,X-SC,2.20,86.0,-0.76,0.37,24.22,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.18,3.97,25.97,30.97,47721.0,7017.0,183755.0,-20.46,58.0,X-MC,2.21,75.0,0.15,1.35,17.03,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,PGHH,17907.65,-0.14,-6.11,42.46,33.76,80058.0,-12270.0,188550.0,-33.72,29.0,X-MC,5.07,57.0,-0.15,1.38,0.00,X40,ATH,FMCG
55,QUESS,424.00,-1.05,-32.72,112.31,42.84,49120.0,-21270.0,43736.0,-55.92,32.0,X-SC,37.31,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
9,BAJAJHFL,181.50,0.21,-21.68,89.56,48.45,140636.0,-43476.0,157030.0,-27.39,13.0,X-MC,10.56,64.0,-0.31,1.15,0.21,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.32,-37.83,117.46,35.20,93968.0,-48670.0,80000.0,4.84,26.0,X-SC,14.98,92.0,-0.52,0.59,0.37,X40,NTT,FOOTWEAR
75,TMPV,600.00,0.27,-15.98,69.23,42.18,160773.0,-44180.0,232230.0,-24.30,31.0,X-LC,4.45,3.0,-0.27,1.70,0.60,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.05,-32.72,112.31,42.84,49120.0,-21270.0,43736.0,-55.92,32.0,X-SC,37.31,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-0.75,-79.82,518.52,24.79,263476.0,-201043.0,50813.0,-80.65,47.0,X-SC,2.20,86.0,-0.76,0.37,24.22,X40N,NTT,MISC
59,RELAXO,1176.00,0.12,-47.55,190.37,52.31,144948.0,-69020.0,76140.0,-44.15,36.0,X-SC,5.69,91.0,-0.48,0.56,1.92,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.32,-37.83,117.46,35.20,93968.0,-48670.0,80000.0,4.84,26.0,X-SC,14.98,92.0,-0.52,0.59,0.37,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.25,-11.92,38.04,21.58,46955.0,-16704.0,123435.0,-55.63,26.0,X-MC,5.33,56.0,-0.36,0.91,9.28,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.48,-11.59,34.92,19.28,106802.0,-40108.0,305848.0,-22.98,71.0,X-LC,6.04,1.0,-0.38,2.25,13.07,X40,ATH,IT
41,INFY,2275.00,0.43,10.49,40.16,54.87,138841.0,32832.0,345720.0,-13.85,71.0,X-LC,3.41,2.0,0.24,2.54,19.64,X40,BTT,IT
75,TMPV,600.00,0.27,-15.98,69.23,42.18,160773.0,-44180.0,232230.0,-24.30,31.0,X-LC,4.45,3.0,-0.27,1.70,0.60,XY24,NTT,AUTO
81,VBL,671.64,-0.50,-3.53,40.64,35.67,123821.0,-11165.0,304677.0,-14.99,57.0,X-LC,4.89,4.0,-0.09,2.24,9.83,X40N,ATH,FMCG
43,ITC,452.00,-0.17,-1.23,11.81,10.44,23346.0,-2460.0,197678.0,-41.15,48.0,X-LC,1.57,5.0,-0.11,1.45,4.27,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.17,-35.87,107.22,32.89,53088.0,-27694.0,49513.0,-711.53,65.0,L-MC,6.46,259.0,-0.52,0.36,36.36,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.69,-17.78,120.97,81.67,93302.0,-16682.0,77128.0,6927.78,36.0,L-SC,15.59,271.0,-0.18,0.57,49.36,XR,NTT,CERAMICS
50,MASFIN,398.61,1.65,-0.21,22.31,22.05,21812.0,-210.0,97770.0,-14.18,61.0,H-SC,7.04,164.0,-0.01,0.72,42.12,XR,ATH,FINANCE
13,BSOFT,831.70,0.06,-21.50,92.08,50.79,101670.0,-30238.0,110415.0,0.89,73.0,H-SC,5.60,151.0,-0.30,0.81,28.56,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.11,-12.13,13.83,0.02,21212.0,-21183.0,153376.0,133.82,53.0,M-SC,16.90,234.0,-1.00,1.13,33.41,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.06,-21.50,92.08,50.79,101670.0,-30238.0,110415.0,0.89,73.0,H-SC,5.60,151.0,-0.30,0.81,28.56,XR,ATH,IT
5,ASIANPAINT,3460.25,-0.36,3.38,16.99,20.95,44223.0,8518.0,260286.0,2.32,77.0,X-LC,18.93,31.0,0.19,1.91,40.23,X40,ATH,PAINTS
32,HCLTECH,1922.01,0.48,9.74,13.66,24.73,36266.0,23563.0,265487.0,14.46,72.0,X-LC,5.55,7.0,0.65,1.95,26.46,X40,ATH,IT
84,WIPRO,420.00,0.86,7.52,60.22,72.27,109713.0,12739.0,182187.0,-7.89,75.0,M-LC,2.79,99.0,0.12,1.34,14.16,XR,NTT,IT
40,INDUSINDBK,1800.00,-0.17,-35.87,107.22,32.89,53088.0,-27694.0,49513.0,-711.53,65.0,L-MC,6.46,259.0,-0.52,0.36,36.36,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.84
1,25,45.78
2,50,76.64


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.03
MC    30.16
LC    24.84
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.32
X40      22.45
X40N     10.87
XR       10.09
AR        9.39
XY25      9.35
OX40N     7.75
SR        1.03
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.55
X-MC    23.43
X-LC    21.25
M-SC    12.14
X-SC     6.93
H-MC     4.95
M-MC     1.42
L-SC     1.41
M-LC     1.34
H-LC     1.22
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.37,-6.30,40.67
IT,12.86,-15.09,77.31
FINANCE,9.61,-18.47,68.54
ELECTRICAL,6.27,-9.91,50.17
PAINTS,6.19,-9.62,25.90
MISC,6.12,-55.61,119.36
INSURANCE,4.54,-2.24,38.27
PHARMA,4.06,-0.82,33.08
AUTO,3.44,-22.15,74.52


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3179073.0,21
AR,1291667.0,10
XR,1289085.0,13
X40,979962.0,14
X40N,968038.0,9
OX40N,737830.0,10
XY25,367127.0,6
SR,281133.0,2
MH,78349.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3636179.0,25
M-SC,1405845.0,15
X-MC,1290032.0,16
X-SC,975744.0,8
X-LC,868013.0,11
H-MC,404151.0,3
L-SC,270117.0,3
M-LC,109713.0,1
H-LC,67933.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1276914.0      6
           AR         888373.0      5
M-SC       XY24       798404.0      6
H-SC       XR         771426.0      7
X-SC       X40N       502592.0      3
X-MC       X40        449816.0      7
           XY24       384435.0      3
X-LC       X40        349478.0      5
H-SC       OX40N      339984.0      4
M-SC       OX40N      309852.0      5
X-SC       XY24       292484.0      3
H-SC       SR         281133.0      2
X-MC       X40N       269212.0      4
X-LC       XY24       233192.0      2
H-MC       AR         210507.0      2
X-LC       X40N       196234.0      2
H-MC       XY24       193644.0      1
X-MC       XY25       186569.0      2
L-SC       XR         182123.0      2
X-SC       X40        180668.0      2
M-SC       XR         172735.0      2
           AR         124854.0      2
M-LC       XR         109713.0      1
X-LC       XY25        89109.0      2
L-SC       OX40N       87994.0      1
H-SC       MH          78349.0      1
H-LC       AR          67933.0      1
L-MC       XR          53088.0      1
M-MC       XY25        52165.0      1
L-LC       XY25        39284.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
